In [1]:
#GUI
#Webscraping from cricbuzz to get all the score
#Display the live match updates on my python app

In [2]:
#pip install pillow

In [1]:
from tkinter import *
from tkinter.ttk import Combobox #For dropdown menu
from PIL import ImageTk#To read images for background, we need to import PIL=pillow
from bs4 import BeautifulSoup
import requests
import re

In [2]:
class CricketScore:
    #Constructor 
    def __init__(self, rootWindow): #All score is going to be in rootWindow, so we are adding it in constructor
        self.rootWindow = rootWindow #self = referencing object of a class 
        self.rootWindow.title("LIVE CRICKET SCORE")
        self.rootWindow.geometry('800x500')
        self.bg = ImageTk.PhotoImage(file="Cricket.jpg") #bg = background
        bg = Label(self.rootWindow, image=self.bg).place(x=0,y=0)

        #Let's give a label to my GUI
        self.label = Label(self.rootWindow, text='Live Matches', font=("times new roman",60), compound='center').pack(padx=100, pady=50)

        #Fetch the live match details and set it in my GUI
        self.var = StringVar()
        self.matches = self.match_details()
        self.data = [ i for i in self.matches.keys()]
        self.cb = Combobox(self.rootWindow, values = self.data, width = 50)
        self.cb.place(x=  250, y = 200)

        #Button to check the details of my match
        self.b1 = Button(self.rootWindow, text="Check Score", font=("times new roman", 15), command = self.show_match_details).place(x=50, y=380)

    #Creating command for the check score button
    def select(self):
        return self.cb.get()

    #Show match details
    def show_match_details(self):
        #Building the match detail frame
        self.frame1 = Frame(self.rootWindow, bg="#ADD8E6")
        self.frame1.place(x=180, y=280, width= 600, height =200)

        #Fetching details of the match
        x = self.matches[self.select()]

        #Displaying team names
        Label(self.frame1, text=self.select() + "-" + x['match_header'], font=("times new roman", 15,"bold"), bg="#ADD8E6",
              bd=0).place(x=150, y=15)

        #Displaying details of the match
        Label(self.frame1, text="Score Details: ", font=("times new roman", 10,"bold"), bg="#ADD8E6", fg="black",
              bd=0).place(x=10, y=40)
        Label(self.frame1, text=x['score card'], font=("times new roman", 10,"bold"), bg="#ADD8E6", fg="black",
              bd=0).place(x=20, y=60)
        Label(self.frame1, text="Summary: ", font=("times new roman", 10,"bold"), bg="#ADD8E6", fg="black",
              bd=0).place(x=10, y=100)
        Label(self.frame1, text=x['summary'], font=("times new roman", 10,"bold"), bg="#ADD8E6", fg="black",
              bd=0).place(x=20, y=120)
    
    #We will load all the matches details
    def match_details(self):

        #To scrap the data
        details = self.scrap() #it will details of all the score cards

        #Need to parse and read the relavant data
        live_match = {}
        for detail in details:
            live_team_details = {}
            summary = self.match_summary(detail)
            if summary is not None :
                match_header = self.match_header(detail).text
                teams = self.teams_name(detail)
                score_card = self.team_score(detail)
                live_team_details['summary'] = summary.text
                live_team_details['match_header'] = match_header
                live_team_details['score card'] = score_card[0]+ "::" +score_card[1]
                live_match[teams[0] + "vs" + teams[1]] = live_team_details
        
        return live_match

    #Function to fetch team scores
    def team_score(self, detail):
        t = []
        team1_details = detail.find("div", class_ = "cb-hmscg-bat-txt").text
        team2_details = detail.find("div", class_ = "cb-hmscg-bwl-txt").text
        t.append(team1_details)
        t.append(team2_details)
        return t

    #Function to fetch the team names
    def teams_name(self, detail):
        t = []
        team1_details = detail.find("div", class_="cb-hmscg-bat-txt").text #"\d" locates where first digit comes
        team1_index = re.search(r"\d", team1_details).start() if re.search(r"\d", team1_details) else len(team1_details)
        
        team2_details = detail.find("div", class_="cb-hmscg-bwl-txt").text 
        team2_index = re.search(r"\d", team2_details).start() if re.search(r"\d", team2_details) else len(team2_details)

        # ERROR HERE IN TEAM 2 ! 

        t.append(team1_details[:team1_index])
        t.append(team2_details[:team2_index])
        return t

    #Function to fetch a given match summary
    def match_summary(self, detail):
        return detail.find("div", class_="cb-mtch-crd-state")
    
    #Function to fetch match header
    def match_header(self,detail):
        return detail.find("div", class_="cb-mtch-crd-hdr")

    #Function to scrap the website cricbuzz
    def scrap(self):
        URL = "https://www.cricbuzz.com/"
        page = requests.get(URL) #gets all html content
        #parse this response and fetch the relevant data
        soup = BeautifulSoup(page.content, "html.parser")
        results = soup.find(id="match_menu_container")
        scrap_results = results.find_all("li", class_="cb-match-card")
        return scrap_results    

#Main function to start application
def main():
    #Start the GUI Window
    rootWindow = Tk()#refer gpt for tk()
    
    #Fetch the cricket score
    obj = CricketScore(rootWindow)

    #Keep on displaying the GUI Window to fetch the live updates
    rootWindow.mainloop()

#Execute this file
if __name__ == '__main__': #..name.. , execute file , execute file --> part wala: name = main
    main()